# Selenium-Scraper Eidgenössische Steuerverwaltung

Mit diesem Scraper wird die interaktive Steuerbelastungsstatistik der Eidgenössischen Steuerverwaltung (ESTV) gescrapt: 
https://swisstaxcalculator.estv.admin.ch/#/taxburden/income-wealth-tax

Weil die Seite interaktiv ist, wird Selenium eingesetzt. Das Ziel ist es, für jede Aargauer Gemeinde für fünf verschiedene Haushaltstypen und Einkommen die Steuerbelastungen in Prozent für die Jahre 2016 bis 2020 als Excel-Datei (einfachste Möglichkeit) herunterzuladen. Für die Haushaltstypen und Einkommen orientiere ich mich am ESTV-Bericht "Unterschiede in der Steuerbelastung natürlicher und juristischer Personen 2004 – 2016".

# Vorgehen

Auf der Website müssen mehrere Optionen aus Buttons und Dropdowns gewählt werden. Dazu wird im Folgenden immer die Funktion find_element_by_xpath benutzt. An einer Stelle muss die Gemeinde eingegeben werden. Jene Gemeinden, die nicht funktionieren, werden in einer csv-Datei gespeichert, damit sie später von Hand heruntergeladen werden können.

In [ ]:
from selenium import webdriver
from time import sleep
import pandas as pd
import csv

Um eine Liste der Aargauer Gemeinden zu erstellen, wird zunächst eine Liste der Aargauer Gemeinden importiert. Daraus werden alle Gemeinden (BFS-Nummer grösser als 4000) benötigt. Jede Gemeinde wird einmal (unique) in der Liste 'gemeindeliste' abgelegt. Weil es sich um eine Liste des Jahres 2017 handelt, hat es auch Gemeinden darunter, die wegen Fusionen nicht mehr existieren. Diese werden in der Liste der fehlerhaften Gemeinden gelistet, sind aber nicht weiter zu beachten.

In [ ]:
df = pd.read_csv('Daten/kanton_ag_gemeindestand.csv', encoding='latin', delimiter=';')

df = df[df['BFSNR'] > 4000].copy()
gemeindeliste = df['NAME'].unique().tolist()

Aus dem ersten Durchlauf mit einem Haushaltstyp ist bekannt, welche Gemeinden auf der ESTV-Seite nicht gefunden wurden. Diese werden im nächsten Schritt entfernt und durch die Namen ersetzt, unter der die Gemeinden tatsächlich zu finden sind.

In [ ]:
gemeinden_entfernen = ['Rudolfstetten-Friedlisberg', 'Schinznach-Bad', 'Schinznach-Dorf', 'Schinznach', 'Möriken-Wildegg']
gemeinden_hinzufügen = ['Rudolfstetten', 'Schinznach Bad', 'Schinznach Dorf', 'Möriken AG']

for g in gemeinden_entfernen:
    gemeindeliste.remove(g)

for g in gemeinden_hinzufügen:
    gemeindeliste.append(g)

In [ ]:
gemeindeliste[0:6]

In [ ]:
len(gemeindeliste)

Nun zum eigentlichen Scraper. In der csv-Datei werden jene Gemeinden erfasst, für die eine Fehlermeldung kommt. Dies wird gemacht, um jene Gemeinden zu sammeln, die in der ESTV-Liste nicht vorkommen, die also später von Hand heruntergeladen werden müssen. Es handelt sich dabei nur um eine, wie sich nach einem ersten ganzen Durchlauf zeigt: Erlinsbach (AG). Weitere Gemeinden werden gelistet, existieren aber heute nicht mehr (wegen Gemeindefusionen) und sind darum irrelevant.

Für jeden Haushaltstyp und jede Gemeinde (innerer Loop) wird die Seite aufgerufen.

In [ ]:
haushaltstypen_xpath = {'ledig': '//*[@id="mass-export-model"]/div[2]/select/option[2]',
                        'vereiratet_o_kinder': '//*[@id="mass-export-model"]/div[2]/select/option[7]',
                       'verheiratet_m_kinder': '//*[@id="mass-export-model"]/div[2]/select/option[9]',
                       'rentner': '//*[@id="mass-export-model"]/div[2]/select/option[24]'
}

with open('gemeindeliste.csv', 'a+') as file:
    csv_writer = csv.writer(file)
    
    for haushaltstyp, xpath in haushaltstypen_xpath.items():
        
        for gemeinde in gemeindeliste:

            #Chromedriver starten, Seite aufrufen
            driver = webdriver.Chrome()
            driver.get('https://swisstaxcalculator.estv.admin.ch/#/taxburden/income-wealth-tax')
            sleep(2)

            #Steuersubjekt aus Dropdown auswählen
            driver.find_element_by_xpath(xpath).click()

            #Konfession = 'keine/andere' anklicken
            driver.find_element_by_xpath('//*[@id="CONFESSION"]/div[2]/div/div[4]').click()

            #Vergleich über Steuerjahre anklicken
            driver.find_element_by_xpath('//*[@id="STATISTIC_TYPE"]/div[2]/div/div[1]/div').click()

            #2016
            driver.find_element_by_xpath('//*[@id="YEARS"]/div[2]/div/div/div[3]/div').click()
            #2017
            driver.find_element_by_xpath('//*[@id="YEARS"]/div[2]/div/div/div[4]/div').click()
            #2018
            driver.find_element_by_xpath('//*[@id="YEARS"]/div[2]/div/div/div[5]/div').click()
            #2019
            driver.find_element_by_xpath('//*[@id="YEARS"]/div[2]/div/div/div[6]/div').click()
            #2020
            driver.find_element_by_xpath('//*[@id="YEARS"]/div[2]/div/div/div[7]/div').click()

            #Einkommen
            driver.find_element_by_xpath('//*[@id="INCOMES-text"]/div').click()

            sleep(2)

            #50k
            driver.find_element_by_xpath('//*[@id="KLASSE"]/div[2]/div[2]/div/div[8]/div').click()
            #100k
            driver.find_element_by_xpath('//*[@id="KLASSE"]/div[2]/div[2]/div/div[13]/div').click()
            #200k
            driver.find_element_by_xpath('//*[@id="KLASSE"]/div[2]/div[2]/div/div[17]/div').click()
            #500k
            driver.find_element_by_xpath('//*[@id="KLASSE"]/div[2]/div[2]/div/div[21]/div').click()
            #1000k
            driver.find_element_by_xpath('//*[@id="KLASSE"]/div[2]/div[2]/div/div[23]/div').click()

            #Gemeinde. Zunächst muss das Feld angeklickt werden
            driver.find_element_by_xpath('//*[@id="tax-group-type-TAX_LOCATION"]/div').click()
            sleep(1)

            #nach einer kurzen Pause kann der Gemeindename eingegeben werden
            driver.find_element_by_xpath('//*[@id="downshift-0-input"]').send_keys(gemeinde)
            sleep(2)

            try:
                #hier wird die Gemeinde angewählt
                gemeinde_wählen = driver.find_element_by_xpath('//*[@id="downshift-0-item-0"]')
                sleep(2)
                gemeinde_wählen.click()

                #Art der Ergbenisse: Belastung in %
                driver.find_element_by_xpath('//*[@id="OUTPUT"]/div[2]/div/div[2]/div').click()

                #Berechnen
                driver.find_element_by_xpath('//*[@id="page-content"]/div/div/div/form/div[7]/button[1]/div[1]').click()

                sleep(7)

                #weiterer Try-/Except-Block, falls die Seite mit den Ergebnissen nicht geladen hat
                try:
                    #Export anklicken
                    driver.find_element_by_xpath('/html/body/div[1]/div[4]/main/div/div/div[2]/div/div[6]/button[1]').click()
                    sleep(2)
                    driver.close()
                    sleep(1)
                except:
                    #in die Datei schreiben, falls der Download nicht gelungen ist
                    csv_writer.writerow([gemeinde, 'Download nicht gelungen'])
                    driver.close()
                    pass
            except:
                #in die Datei schreiben, falls die Gemeinde nicht gefunden wurde
                csv_writer.writerow([gemeinde])
                driver.close()
                pass       

Es gilt zu beachten: Wenn eine Gemeinde mehrere Postleitzahlen hat, wird mit diesem Scraper nur die erste abgegriffen (zB. Baden und Aarau). Ein Vergleich der Aarauer Postleitzahlen zeigt, dass der Steuerfuss für alle der gleiche ist.